<a href="https://colab.research.google.com/github/datanicaragua/Tools/blob/main/UTMPolygonizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UTMPolygonizer

# Versión: 1.0
# Autor: Gustavo Ernesto Martinez Cardenas
# https://datanicaragua.com
# Licencia: [MIT](https://choosealicense.com/licenses/mit/)

---

# **GENERAR POLIGONO A PARTIR DE PUNTOS **
*(Waypoints to Polygons)*: CSV/Excel to Geojson

# **INSTRUCCIONES**

# PASO 1: FORMATO

Antes de cargar su archivo con puntos de waypoint en formato UTM, asegúrese de que cumpla con el siguiente formato:

La tabla debe contener las siguientes columnas: Vertice, Easting, Northing, Zone.

1. Vertice: debe contener números enteros en orden secuencial, comenzando en 1 y terminando en n, donde 1 es el punto de inicio y n es el punto final.
2. Easting (X): debe contener las coordenadas este en metros.
3. Northing (Y): debe contener las coordenadas norte en metros.
4. Zone: debe contener la zona UTM, que para Nicaragua es la 16N. Esta misma zona es utilizada en parte de Centroamérica y otras regiones que estén dentro del mismo rango longitudinal de la zona 16N del sistema UTM.

Un ejemplo de cómo deben lucir sus datos en el archivo es el siguiente:

| Vertice | Easting | Northing | Zone |
|---------|---------|----------|------|
| 1       | 500000  | 1300000  | 16N  |
| 2       | 500050  | 1300050  | 16N  |
| ...     | ...     | ...      | ...  |
| n       | 500100  | 1300100  | 16N  |



Asegúrese de que su archivo está en formato CSV o Excel antes de subirlo.

# PASO 2: LIBRERIAS
Haga clic para ejecutar la instalación de librerías y el entorno necesario

In [ ]:
# @title
!pip install geopandas

import io
from io import BytesIO  # Asegurarse de incluir este import
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from google.colab import files

# PASO 3: UPLOAD FILE
Suba o cargue el archivo CSV o Excel.

In [ ]:
# @title
uploaded = files.upload()

# Función para procesar el archivo subido
def process_uploaded_file(uploaded):
    for filename, content in uploaded.items():
        if filename.endswith('.csv'):
            df = pd.read_csv(BytesIO(content))
        elif filename.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(BytesIO(content))
        else:
            raise ValueError("El archivo proporcionado no es ni CSV ni Excel.")
        return df

df = process_uploaded_file(uploaded)

KeyboardInterrupt: 

# PASO 4: GENERAR POLIGONO

In [ ]:
# Define la proyección UTM Zona 16N
utm_zone_16n = "EPSG:32616"

# Procesa los datos para crear el GeoDataFrame
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Easting, df.Northing),
    crs=utm_zone_16n
)

# Genera el polígono a partir de los puntos
polygon = Polygon([(point.x, point.y) for point in gdf_points.geometry])

# Crea un GeoDataFrame para el polígono
gdf_polygon = gpd.GeoDataFrame(index=[0], crs=utm_zone_16n, geometry=[polygon])

# Calcula estadísticas para el polígono UTM
area_sq_meters = polygon.area
area_hectares = area_sq_meters / 10000
perimeter_meters = polygon.length
minx, miny, maxx, maxy = polygon.bounds
centroid = polygon.centroid

# Agrega estadísticas al GeoDataFrame del polígono
gdf_polygon.at[0, 'Area_sq_m'] = area_sq_meters
gdf_polygon.at[0, 'Area_ha'] = area_hectares
gdf_polygon.at[0, 'Perimeter_m'] = perimeter_meters
gdf_polygon.at[0, 'Min_X'] = minx
gdf_polygon.at[0, 'Max_X'] = maxx
gdf_polygon.at[0, 'Min_Y'] = miny
gdf_polygon.at[0, 'Max_Y'] = maxy
gdf_polygon.at[0, 'Centroid_X'] = centroid.x
gdf_polygon.at[0, 'Centroid_Y'] = centroid.y

# Imprimir estadísticas
print("Estadísticas del polígono en UTM:")
print(f"Área: {area_sq_meters} m² ({area_hectares} hectáreas)")
print(f"Perímetro: {perimeter_meters} m")
print(f"Límites: Mínimo X (Este): {minx} m, Máximo X (Este): {maxx} m")
print(f"Mínimo Y (Norte): {miny} m, Máximo Y (Norte): {maxy} m")
print(f"Centroide: Este {centroid.x}, Norte {centroid.y}")

# Combina puntos y polígonos
gdf_combined = pd.concat([gdf_points, gdf_polygon], ignore_index=True)

# Convertir el GeoDataFrame UTM a WGS84 para la exportación a GeoJSON
gdf_combined_wgs84 = gdf_combined.to_crs(epsg=4326)

# Solicitar al usuario que nombre el archivo GeoJSON
print("Por favor, ingrese un nombre para el archivo GeoJSON (sin extensión):")
output_filename = input() + '.geojson'

# Verificar que el usuario ha ingresado un nombre de archivo
if not output_filename.strip('.geojson'):
    output_filename = "output.geojson"

# Exporta a GeoJSON
gdf_combined_wgs84.to_file(output_filename, driver='GeoJSON')

# Proporciona un enlace para descargar el archivo GeoJSON generado
files.download(output_filename)

# Informar al usuario sobre la descarga
print(f"Se está generando el archivo GeoJSON: {output_filename} para su descarga.")
print("El archivo GeoJSON se descargará en la carpeta de descargas predeterminada de su navegador o en la ubicación que haya especificado en la configuración de descargas de su navegador.")



Estadísticas del polígono en UTM:
Área: 14523.5 m² (1.45235 hectáreas)
Perímetro: 548.0820660560939 m
Límites: Mínimo X (Este): 554442.0 m, Máximo X (Este): 554607.0 m
Mínimo Y (Norte): 1311610.0 m, Máximo Y (Norte): 1311766.0 m
Centroide: Este 554526.8032499054, Norte 1311699.144076841
Por favor, ingrese un nombre para el archivo GeoJSON (sin extensión):
